In [2]:
# -*- coding: utf-8 -*-
"""modeling.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1FywCo2ods8bIB6ot9bUIGxGdqekxz7p6
"""
#%%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from tqdm import tqdm

pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)
pd.set_option("display.max_colwidth",None)

#%%

# Load data
train_data = pd.read_csv("train.csv", index_col="listing_id")
test_data = pd.read_csv("test.csv", index_col="listing_id")
train_index = train_data.index
test_index = test_data.index

merge_data = pd.concat([train_data, test_data], axis=0).drop(columns=["price"])

merge_data = merge_data.drop(
    columns=["title", "description", "features", "accessories"]
)

# %%

# Remove those almost na cols.
def remove_high_na_columns(data: pd.DataFrame, threshold=0.3):
    na_rate = data.isna().mean()
    high_na_columns = na_rate[na_rate > threshold]
    print(f"Removing columns with high missing values:\n{high_na_columns}")
    data = data.drop(columns=high_na_columns.index)
    return data


merge_data = remove_high_na_columns(merge_data)
merge_data.info()

# %%

# Remove those cols with all the same values.
def remove_one_value_columns(data: pd.DataFrame):
    duplicate_rate = data.apply(lambda col: col.nunique())
    high_duplicate_columns = duplicate_rate[duplicate_rate == 1]
    print(f"Removing columns with high duplicate values:\n{high_duplicate_columns}")
    data = data.drop(columns=high_duplicate_columns.index)
    return data


merge_data = remove_one_value_columns(merge_data)
merge_data.info()

# %%

# Inspect on reg_date and manufactured, we can see they are highly related. So remove manufactured(has na values).
merge_data["reg_date"] = pd.to_datetime(merge_data["reg_date"], format=r"%d-%b-%Y")
merge_data["reg_year"] = merge_data["reg_date"].dt.year
merge_data[["reg_year", "manufactured"]].corr()


# %%

merge_data = merge_data.drop(columns=["reg_date", "manufactured"])

# %%

# Inspect on make and model, we can find that for those rows which only has model value, we can inference the make.
def inspect_make_model(data: pd.DataFrame):
    data["make"] = data["make"].fillna("unknown")
    model_with_unknown_make = data.loc[data["make"] == "unknown", "model"].unique()
    for model in model_with_unknown_make:
        print(f"{model}: {data.loc[data['model'] == model, 'make'].unique()}")


inspect_make_model(merge_data)

# %%

# Inspect on make and model, we can find that for those rows which only has model value, we can inference the make.
def inspect_make_model(data: pd.DataFrame):
    data["make"] = data["make"].fillna("unknown")
    model_with_unknown_make = data.loc[data["make"] == "unknown", "model"].unique()
    for model in model_with_unknown_make:
        print(f"{model}: {data.loc[data['model'] == model, 'make'].unique()}")


inspect_make_model(merge_data)

# %%

# fill missing make value


def process_make(data: pd.DataFrame):
    data["make"] = data["make"].fillna("unknown")

    model_with_unknown_make = data.loc[data["make"] == "unknown", "model"].unique()
    model_make_map = (
        data[
            (data["model"].isin(model_with_unknown_make)) & (data["make"] != "unknown")
        ]
        .drop_duplicates("model")
        .set_index("model")["make"]
        .to_dict()
    )
    loc_to_fill = data["make"] == "unknown"
    data.loc[loc_to_fill, "make"] = data.loc[loc_to_fill, "model"].map(model_make_map)

    print("Filled missing values in make column")

    return data


merge_data = process_make(merge_data)
merge_data.info()

# %%

# Notice that type_of_vehicle, category, transmission three variables are like tags, extract and convert them to dummies.


def process_type_of_vehicle(data: pd.DataFrame):
    type_of_vehicle = data["type_of_vehicle"].str.get_dummies()
    data = pd.concat([data, type_of_vehicle], axis=1)
    data = data.drop(columns=["type_of_vehicle"])
    print(f"type_of_vehicle col is extracted to {len(type_of_vehicle.columns)} cols")
    return data


def process_category(data: pd.DataFrame):
    category = data["category"].str.get_dummies(sep=", ")
    data = pd.concat([data, category], axis=1)
    data = data.drop(columns=["category"])

    print(f"category col is extracted to {len(category.columns)} cols")

    return data


def process_transmission(data: pd.DataFrame):
    transmission = data["transmission"].str.get_dummies()
    data = pd.concat([data, transmission], axis=1)
    data = data.drop(columns=["transmission"])
    print(f"transmission col is extracted to {len(transmission.columns)} cols")
    return data


def process_make_model(data: pd.DataFrame):
    make = data["make"].str.get_dummies()
    data = pd.concat([data, make], axis=1)
    data = data.drop(columns=["make", "model"])
    print(f"make col is extracted to {len(make.columns)} cols")
    return data


merge_data = process_type_of_vehicle(merge_data)
merge_data = process_category(merge_data)
merge_data = process_transmission(merge_data)

# make 转或者不转 dummy,影响后边的内容
merge_data = merge_data.drop(columns=['make', 'model'])
# merge_data = process_make_model(merge_data)
merge_data.info()

# %%

# 年平均行驶里程超过 10w 英里，真有点逆天了吧
high_mileage_indices = merge_data[
    ((merge_data["mileage"] / (2025 - merge_data["reg_year"])) > 1e5)
].index
train_data.loc[high_mileage_indices][
    ["make", "model", "manufactured", "mileage", "price"]
]
merge_data = merge_data.drop(high_mileage_indices)

# %%

merge_data.info()

# %%

# Begin to fill numeric na values
numeric_feats = merge_data.select_dtypes(include=[np.number]).columns
dummy_feats = merge_data.select_dtypes(include=[bool]).columns


from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# 定义插补方法
iterative_imputer = IterativeImputer(
    max_iter=20, random_state=0, keep_empty_features=True, verbose=1
)


def custom_impute_by_make(data):
    # Step 1: 按 (make, model) 组内插补
    for (make, model), group in data.groupby(["make", "model"]):
        if len(group) >= 10:  # 检查组大小
            imputed_values = iterative_imputer.fit_transform(group[numeric_feats])
            data.loc[group.index, numeric_feats] = imputed_values

    # Step 2: 对于尚未填补的缺失值，按 make 组插补
    for make, group in data.groupby("make"):
        if len(group) >= 10:  # 检查组大小
            missing_indices = group[group.isnull().any(axis=1)].index
            if not missing_indices.empty:
                data.loc[group.index, numeric_feats] = iterative_imputer.fit_transform(
                    group[numeric_feats]
                )

    # Step 3: 使用 全局 插补剩余缺失值
    remaining_na_indices = data[data.isnull().any(axis=1)].index
    if not remaining_na_indices.empty:
        data.loc[remaining_na_indices, numeric_feats] = iterative_imputer.fit_transform(
            group[numeric_feats]
        )

    return data



# 层次化插值还是直接插值补充
data_imputed = pd.DataFrame(
    iterative_imputer.fit_transform(merge_data),
    columns=merge_data.columns,
    index=merge_data.index,
)

# %%
data_imputed.head()

valid_train_index = train_index.intersection(data_imputed.index)
X = data_imputed.loc[valid_train_index]
y = train_data.loc[valid_train_index]['price']

# X.corrwith(y).sort_values(ascending=False)
selected_feats = X.corrwith(y) > 0.05
selected_feats = selected_feats[selected_feats].index

X = X[selected_feats]

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from tqdm import tqdm

def rf(X, y):

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    y_pred = []
    too_less = []
    for i in tqdm(range(len(X_test))):

        rf_model = RandomForestRegressor(random_state=42, n_jobs=-1)

        row = X_test.iloc[i]

        interest_index = (X_train["make"] == row["make"]) & (
            X_train["model"] == row["model"]
        )

        # Step 1: 按 (make, model) 拟合
        if interest_index.sum() > 10:
            rf_model.fit(X_train[interest_index].drop(columns=['make', 'model']), y_train[interest_index])

        else:
            interest_index = X_train["make"] == row["make"]
            # Step 2: 按 make 拟合
            if interest_index.sum() > 10:
                rf_model.fit(X_train[interest_index].drop(columns=['make', 'model']), y_train[interest_index])

            # Step 3: 使用 全局 拟合
            else:
                too_less.append(1)
                rf_model.fit(X_train.drop(columns=['make', 'model']), y_train)

        # 预测和计算 RMSE
        y_pred.append(rf_model.predict(pd.DataFrame([row.drop(['make', 'model'])]))[0])
    print(sum(too_less))
    mse = mean_squared_error(y_test, y_pred)
    print(f"RF RMSE: {mse**0.5}")

    return rf_model


# rf_model1 = rf(X, y)

# %%

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from tqdm import tqdm

valid_train_index = train_index.intersection(data_imputed.index)
X = data_imputed.loc[valid_train_index]
y = train_data.loc[valid_train_index]['price']

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
y_pred = lr_model.predict(X_test)
print(f"LR RMSE: {mean_squared_error(y_test, y_pred)**0.5}")

# %%

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
rf_model = RandomForestRegressor(n_estimators=300, n_jobs=-1, verbose=1)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
print(f"RF RMSE: {mean_squared_error(y_test, y_pred)**0.5}")

# %%

from sklearn.linear_model import Lasso
from sklearn.preprocessing import MinMaxScaler
# 归一化数据
scaler = MinMaxScaler()

X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

lasso_model = Lasso(alpha=0.01,max_iter=100000)
lasso_model.fit(X_train_scaled, y_train)
y_pred = lasso_model.predict(X_test_scaled)

print(f"LASSO RMSE: {mean_squared_error(y_test, y_pred)**0.5}")
selected_data = X_train[X_train.columns[lasso_model.coef_ != 0]]



# %%

# y_pred = rf_model.predict(merge_data.loc[test_index].drop(columns=["make", "model"]))


def save_submission(y_pred, filename="submission.csv"):
    output = pd.DataFrame({"Id": range(len(y_pred)), "Predicted": y_pred})
    output.to_csv(filename, index=False)


# save_submission(y_pred, "submission_rf_trained_all.csv")

Removing columns with high missing values:
original_reg_date    0.989943
fuel_type            0.764886
opc_scheme           0.994029
lifespan             0.905629
indicative_price     1.000000
dtype: float64
<class 'pandas.core.frame.DataFrame'>
Index: 35000 entries, 1292132 to 1311711
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   make             33143 non-null  object 
 1   model            35000 non-null  object 
 2   manufactured     34990 non-null  float64
 3   reg_date         35000 non-null  object 
 4   type_of_vehicle  35000 non-null  object 
 5   category         35000 non-null  object 
 6   transmission     35000 non-null  object 
 7   curb_weight      34583 non-null  float64
 8   power            31274 non-null  float64
 9   engine_cap       34169 non-null  float64
 10  no_of_owners     34974 non-null  float64
 11  depreciation     34292 non-null  float64
 12  coe              35000 non-null

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   23.8s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   59.3s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  1.4min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 300 out of 300 | elapsed:    0.4s finished


RF RMSE: 27362.415061901487
LASSO RMSE: 42670.83452727414
